1. 문서의 내용을 읽는다 (docx2txt)
2. 문서를 쪼갠다 (langchain-text-splitters)
  - 토큰수 초과로 답변을 생성하지 못할수 있고
  - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림
3. 임베딩 -> 벡터 데이터베이스에 저장 (OpenAIEmbeddings, chroma)
4. 질문이 있을 때, 백터 테이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 (similarity_search)

In [23]:
%pip install --upgrade --quiet  docx2txt langchain-community


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [24]:
%pip install -qU langchain-text-splitters


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [25]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1500, # 하나의 청크가 가질수있는 토큰수
  chunk_overlap=200 # 청크 간 중복 토큰수
)

loader = Docx2txtLoader("./tax.docx")

document_list = loader.load_and_split(text_splitter=text_splitter)

In [26]:
len(document_list)

183

In [27]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

# 환경변수를 불러옴
load_dotenv()

# Upstage 에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = UpstageEmbeddings(model='embedding-query')

In [28]:
%pip install langchain-chroma


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [29]:
from langchain_chroma import Chroma

# ./chroma에 저장된 내용 가져와서 필요없는건 주석

#database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name="chroma_tax" persist_directory="./chroma")
database = Chroma(collection_name="chroma_tax", persist_directory="./chroma", embedding_function=embedding)

In [30]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
#retrieved_docs = database.similarity_search(query, k=3)

In [31]:
# retrieved_docs

In [32]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage(model="solar-pro2")


In [33]:
# prompt = f"""[Identity]
# - 당신은 최고의 소득세 전문가 입니다
# -[Context]를 참고해서 사용자의 질문에 답변해주세요

# [Context]
# {retrieved_docs}

# Question: {query}
# """

In [34]:
# ai_message = llm.invoke(prompt)

In [35]:
#ai_message.content

In [36]:
%pip install -U langchain langchainhub --quiet


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [37]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

/Users/gim-eung-gil/.pyenv/versions/3.13.5/envs/inflearn-llm-application/lib/python3.13/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [38]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [39]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
  llm,
  retriever=database.as_retriever(),
  chain_type_kwargs={"prompt": prompt}
)

In [40]:
ai_message = qa_chain({"query": query})

/var/folders/5f/xcxvkkgj48g7kftgg_gb86dm0000gn/T/ipykernel_40846/3455095564.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ai_message = qa_chain({"query": query})


In [41]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '제공된 문맥에는 연봉 5천만원에 대한 소득세 계산 방법이 명시되어 있지 않습니다. 근로소득공제 및 세액공제에 대한 일부 규정이 있으나, 구체적인 세액 산정은 추가 정보가 필요합니다. 따라서 정확한 소득세를 알 수 없습니다.'}